In [27]:
import pandas as pd
from hazm import Normalizer, word_tokenize, Stemmer, Lemmatizer, stopwords_list
import re

In [2]:
df = pd.read_csv("./golestan.csv")

print(df.shape)
print(df.columns)


(290, 2)
Index(['hekayt', 'bab'], dtype='object')


In [3]:
df.head()

,hekayt,bab
0,پادشاهی را شنیدم به کُشتنِ اسیری اشارت کرد. بی...,1
1,یکی از ملوکِ خراسان محمودِ سبکتگین را به خواب ...,1
2,ملک‌زاده‌ای را شنیدم که کوتاه بود و حقیر و دیگ...,1
3,طایفهٔ دزدانِ عرب بر سرِ کوهی نشسته بودند و من...,1
4,سرهنگ‌زاده‌ای را بر در سرایِ اُغْلُمُش دیدم که...,1


In [4]:
df = df.dropna(subset=["hekayt"])
df = df.dropna(subset=["bab"])

print(df.shape)
print(df.columns)

(290, 2)
Index(['hekayt', 'bab'], dtype='object')


In [5]:
df["bab"].value_counts()

bab
8    109
2     48
1     41
3     28
5     21
7     20
4     14
6      9
Name: count, dtype: int64

In [6]:
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

In [13]:
base_stopwords = set(stopwords_list())

In [22]:
extra_stopwords = {
    "است", "هست", "بود", "شد", "باشد",
    "می", "هم", "را", "که", "با", "از", "برای",
    "ای", "این", "آن", "من", "تو", "ما", "شما", "او",
}

additional_stopwords = set()

for i in extra_stopwords:
    print(f'if {i} is in stopwords list? {i in base_stopwords}')
    if i not in base_stopwords: 
        additional_stopwords.add(i)

additional_stopwords

if شما is in stopwords list? True
if از is in stopwords list? True
if ای is in stopwords list? False
if شد is in stopwords list? True
if ما is in stopwords list? True
if هم is in stopwords list? True
if تو is in stopwords list? True
if را is in stopwords list? True
if او is in stopwords list? True
if این is in stopwords list? True
if برای is in stopwords list? True
if که is in stopwords list? True
if با is in stopwords list? True
if آن is in stopwords list? True
if باشد is in stopwords list? True
if من is in stopwords list? True
if است is in stopwords list? True
if هست is in stopwords list? False
if بود is in stopwords list? True
if می is in stopwords list? True


{'ای', 'هست'}

In [26]:
stopwords = base_stopwords.union(additional_stopwords)

print(len(base_stopwords))
print(len(stopwords))

389
391


In [41]:
def preprocess_hekayat(text):
    text = str(text)

    text = normalizer.normalize(text)

    tokens = word_tokenize(text)
    tokens = [t for t in tokens if re.search(r"[آ-یءئ]", t)]
    tokens = [t for t in tokens if t not in stopwords]

    tokens_stemmed = [stemmer.stem(t) for t in tokens]

    return " ".join(tokens_stemmed)


In [50]:
df["clean_hekayt"] = df["hekayt"].apply(preprocess_hekayat)

In [49]:
df.to_csv("golestan_clean.csv", index=False)
df[["hekayt", "clean_hekayt"]].head()

,hekayt,clean_hekayt
0,پادشاهی را شنیدم به کُشتنِ اسیری اشارت کرد. بی...,پادشاه شنید کشتن اسیر اشار بیچاره حال نومید مل...
1,یکی از ملوکِ خراسان محمودِ سبکتگین را به خواب ...,ملوک خراس محمود سبکتگین خواب چن دید جمله ریخته...
2,ملک‌زاده‌ای را شنیدم که کوتاه بود و حقیر و دیگ...,ملک‌زاده شنید کوتاه حقیر برادر بلند خوب‌رو بار...
3,طایفهٔ دزدانِ عرب بر سرِ کوهی نشسته بودند و من...,طایفه دزد عرب سر کوه نشسته_بودند منفذ کارو بست...
4,سرهنگ‌زاده‌ای را بر در سرایِ اُغْلُمُش دیدم که...,سرهنگ‌زاده سرا اغلم دید عقل کیاست فه فراست زای...
